1. **Calcolo del numero di frasi e di token nel corpus**. Le tre funzioni permettono di:
- leggere il contenuto del corpus e inserirlo in una variabile (`contents`), su cui lavoro per non accedere direttamente al testo
- fare il sentence splitting del testo, le cui frasi finiscono nella variabile `sentences`, con la funzione di nltk `nltk.tokenize.sent_tokenize`
- a partire dalle frasi (estratte nella seconda funzione), questa funzione seleziona i token per ogni frase con una iterazione e li inserisce con l'operazione `.append` in un nuovo array (`tokens_in_sentences`) dove ogni frase è una lista di liste (ognuna al proprio interno ha i suoi token), mentre con l'operazione `.extend` inserisco tutti i token in una unica lista (`all_tokens`)

In [13]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def read_file_content(file):
    with open(file, 'r', encoding="utf8") as infile:
        contents = infile.read()
        return contents

def get_sentences(text):
    sentences = nltk.tokenize.sent_tokenize(text)
    return sentences

def get_tokens(text_sentences):
    all_tokens = []
    tokens_in_sentences = []
    for sentence in text_sentences:
        tokens = nltk.tokenize.word_tokenize(sentence)
        all_tokens.extend(tokens)
        tokens_in_sentences.append(tokens)
    return all_tokens, tokens_in_sentences

2. **Calcolo della Lunghezza media delle frasi in token e della lunghezza media dei token**. Le due funzioni permettono di:
- calcolare, a partire dal numero di frasi nel corpus e dalle frasi tokenizzate, la lunghezza media in token della frase. Iterando sulle frasi tokenizzate, sotto forma di lista di liste, calcolo la lunghezza di ogni frase (=numero di token dentro ognuna) e sommo tutte le lunghezze inserendole in una variabile (`all_sentence_length`). Per fare la media divido questa variabile per tutte le frasi presenti nel corpus (`number_of_sentences`).
- calcolare, a partire dai token del testo e dalla lunghezza del corpus, la lunghezza media dei token. Anche qui calcolo la lunghezza di ogni token sotto forma di lista di liste (in questo caso le lettere), la sommo alle altre inserendola in una variabile (`all_tokens_length`) e divido poi la variabile per il numero di token presenti nel corpus, che equivale alla lunghezza di quest'ultimo. Una cosa in più che è stata aggiunta è una condizione che mi permette, con una approsimazione, di escludere dall'analisi i segni di punteggiatura. In particolare la funzione aggiunge la lunghezza del token alla somma totale delle lunghezze solo se il token è più lungo di 1.

In [14]:
def sentence_length(number_of_sentences, tokens_in_sentences):
    
    all_sentences_length = 0

    for sentence in tokens_in_sentences:
        sentence_length = len(sentence)
        all_sentences_length = all_sentences_length + sentence_length
    
    return (all_sentences_length/number_of_sentences)

def token_length(text_tokens, corpus_length):
    
    all_tokens_length = 0

    for token in text_tokens:
        if(len(token)!=1):
            all_tokens_length = all_tokens_length + len(token)
    return(all_tokens_length/corpus_length)



3. **Calcolo del numero di hapax nei primi 500, 1000 e 3000 token e nell'intero corpus**. Ho utilizzato una unica funzione che permette di calcolare gli hapax in diversi punti del testo a partire dai token del corpus. All'inizio ho specificato i 4 punti del testo in cui voglio calcolare il numero degli hapax. Li metto in un array per poi creare un unico ciclo che, iterando su ognuno dei valori, mi calcola gli hapax del corpus che come lunghezza il valore che sta iterando. Per fare ciò ogni volta aggiorno il mio corpus (creando un `new_corpus`) attraverso l'operazione di slicing (`tokens[:x]`) che prende i valori di una lista fino all'indice `x`. Per trovare gli hapax in ognuno dei miei nuovi corpora utilizzo un ciclo che lavora sulla distribuzione di frequenze dei token nel testo a partire da una funzione predefinita di nltk (`nltk.FreqDist`). Questa funzione crea un dizionario che associa ad ogni token la sua frequenza. Itero su ogni token del dizionario e con una condizione specifico che mi servono i token con frequenza 1, inserendoli poi in un array (`hapax`). Infine aggiungo la lunghezza di ogni raccolta di hapax in un altro array (`hapax_length`), dove avrò i valori che cercavo.

In [15]:
def get_hapax(tokens):
    inc = [500, 1000, 3000, (len(tokens)-1)]
    hapax_length = []
    for x in inc:
        new_corpus = tokens[:x]
        freq = nltk.FreqDist(new_corpus)
        hapax = []
        for token in freq:
            if(freq[token] == 1):
                hapax.append(token)
        hapax_length.append(len(hapax))
    return hapax_length


4.**Dimensione del vocabolario e ricchezza lessicale (TTR), calcolati per porzioni incrementali di 200 token**. La funzione `voc_TTR` prende in input i token di un corpus e con un ciclo `while` calcola vocabolario e TTR (vocabolario/lunghezza del corpus) ogni 200 token. L'indice `i` lo aggiorno quindi di 200 finché non arriva al valore della lunghezza di tutto il corpus e ogni volta cacolo vocabolario e TTR sul nuovo corpus(`new_corpus`), che è 200 token più grande del precedente. Per far sì di avere come risultato una lista di coppie [voabolario, TTR] ho creato due array: in uno inserisco ogni volta i valori del vocabolario e della TTR (dunque ho una sola lista), mentre `complete_voc_TTR_size` è un array che pesca gli ultimi due valori del primo array (`voc_TTR_size`) con `.append` diventando quindi una lista di liste. Per fare ciò ho usato  sul primo array l'operazione di slicing `[-2:]` che prende i valori di un array dal penultimo all'ultimo.

In [16]:
def voc_TTR(tokens):
    i = 200
    voc_TTR_size = []
    complete_voc_TTR_size = []
    while(i<len(tokens)):
        new_corpus = tokens[:i]
        voc_size = len(list(set(new_corpus)))
        TTR = voc_size/len(new_corpus)
        voc_TTR_size.append(voc_size)
        voc_TTR_size.append(TTR)
        complete_voc_TTR_size.append(voc_TTR_size[-2:])
        i = i + 200
    return complete_voc_TTR_size



5.**Numero di lemmi distinti**. Utilizzo due funzioni:
- La prima converte il PoS predefinito di nltk (basato sul Penn Treebank) nel PoS speech corrispondente in WordNet (che ho importato da `nltk.corpus` - vedi la prima cella). In input ho quindi la pos del treebank che, a seconda del suo valore, verrà converita in una PoS di WordNet.
- La seconda prende in input i token del corpus e tramite la funzione `nltk.pos_tag` mi restituisce una lista di coppie [parola, PoS]. A questo punto per ogni coppia della lista, lemmatizzo il token a partire dalla sua PoS con  `WordNetLemmatizer().lemmatize`, una funzione di  `nltk.stem ` che lemmatizza le parole basandosi sul database lessicale WordNet. La funzione può anche ricevere in input la PoS di WordNet corrispondente alla parola per essere più precisa nella lemmatizzazione. Per fare ciò uso la prima funzione sulle PoS predefinite di ogni parola,  convertendole in PoS di WordNet. Ogni volta che lemmatizzo una parola, la inserisco in una variabile e poi in un array che le racchiuderà tutte (`text_lem`). Infine utilizzo  ` set` e  ` list` sull'array per eliminare le occorrenze e trasformarlo in una lista, così da poter poi restituire la sua lunghezza.

In [17]:
def get_wordnet_pos(penn_treebank_pos):
    if penn_treebank_pos.startswith('J'):
        return wordnet.ADJ
    elif penn_treebank_pos.startswith('V'):
        return wordnet.VERB
    elif penn_treebank_pos.startswith('N'):
        return wordnet.NOUN
    elif penn_treebank_pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def voc_lem(tokens):
    text_lem = []
    word_pos_list = nltk.pos_tag(tokens)
    for token, pos in word_pos_list:
        token_lem = WordNetLemmatizer().lemmatize(token, get_wordnet_pos(pos))
        text_lem.append(token_lem)
    voc_lem = list(set(text_lem))
    return len(voc_lem)

La funzione `main` racchiude tutte le funzioni create in precedenza, applicandole ai due testi in input che gli fornisco (`file_1`, `file_2`) e stampando il risultato. 

In [18]:

def main(file_1_path, file_2_path):
    

    #leggo il contenuto del file e lo inserisco in una variabile
    corpus_1 = read_file_content(file_1_path)
    corpus_2 = read_file_content(file_2_path)


    #conto quante frasi e quanti token contengono i due testi con l'operazione len e inserisco i risultati nelle variabili
    text_1_sentences = get_sentences(corpus_1)
    text_1_tokens, tokens_in_sentences_1 = get_tokens(text_1_sentences)

    text_2_sentences = get_sentences(corpus_2)
    text_2_tokens, tokens_in_sentences_2 = get_tokens(text_2_sentences)

    number_of_sentences_1 = len(text_1_sentences)
    corpus_length_1 = len(text_1_tokens)

    number_of_sentences_2 = len(text_2_sentences)
    corpus_length_2 = len(text_2_tokens)

    print(f" - Confronto del numero di frasi e di token dei due corpora:")
    print(" ")
    print (f"   Le frasi presenti nel primo corpus sono {number_of_sentences_1}, mentre i token sono {corpus_length_1}.")
    print(" ")
    print(f"   Le frasi presenti nel secondo corpus sono {number_of_sentences_2}, mentre i token sono {corpus_length_2}.")
    print(" ")


    #calcolo la lunghezza media delle frasi in token
    sentence_length_1 = sentence_length(number_of_sentences_1, tokens_in_sentences_1)
    sentence_length_2 = sentence_length(number_of_sentences_2, tokens_in_sentences_2)


    #calcolo la lunghezza media dei token
    token_length_1 = token_length(text_1_tokens, corpus_length_1)
    token_length_2 = token_length(text_2_tokens, corpus_length_2)

    print(f" - Confronto della lunghezza media delle frasi in token:")
    print(" ")
    print (f"   La lunghezza media delle frasi nel primo corpus è {sentence_length_1}, mentre quella dei token è {token_length_1}.")
    print(" ")
    print(f"   La lunghezza media delle frasi nel secondo corpus è {sentence_length_2}, mentre quella dei token è {token_length_2}.")
    print(" ")


    #calcolo del numero di hapax nei primi 500, 1000, 3000 token e in tutto il corpus.
    hapax_1 = get_hapax(text_1_tokens)
    hapax_2 = get_hapax(text_2_tokens)

    print(f" - Confronto del numero di hapax nei primi 500, 1000, 3000 token e in tutto il corpus:")
    print(" ")
    print(f"    Nel primo corpus gli hapax tra primi 500, 1000, 3000 token e in tutto il corpus sono {hapax_1}.")
    print(" ")
    print(f"    Nel secondo corpus gli hapax tra primi 500, 1000, 3000 token e in tutto il corpus sono {hapax_2}.")
    print(" ")

    #calcolo della dimensione del vocabolario e ricchezza lessicale (TTR) per porzioni incrementali di 200 token
    voc_TTR_size_1 = voc_TTR(text_1_tokens)
    voc_TTR_size_2 = voc_TTR(text_2_tokens)

    print(f" - Confronto della dimensione del vocabolario e della ricchezza lessicale per proporizoni incrementali di 200 token:")
    print(" ")
    print(f"    La dimensione del vocabolario e ricchezza lessicale (TTR), calcolati per porzioni incrementali di 200 token, nel primo corpus è {voc_TTR_size_1}")
    print(" ")
    print(f"    La dimensione del vocabolario e ricchezza lessicale (TTR), calcolati per porzioni incrementali di 200 token, nel secondo corpus è {voc_TTR_size_2}")
    print(" ")

    #calcolo del vocabolario di lemmi
    voc_lem_1 = voc_lem(text_1_tokens)
    voc_lem_2 = voc_lem(text_2_tokens)

    print(f" - Confronto della dimensione del vocabolario di lemmi:")
    print(" ")
    print(f"    La dimensione del vocabolario di lemmi nel primo corpus è di {voc_lem_1} lemmi.")
    print(" ")
    print(f"    La dimensione del vocabolario di lemmi nel secondo corpus è di {voc_lem_2} lemmi.")
    print(" ")



file_1 = "Heart_of_darkness.txt"
file_2 = "Language_Sapir.txt"

main(file_1, file_2)

 - Confronto del numero di frasi e di token dei due corpora:
 
   Le frasi presenti nel primo corpus sono 2191, mentre i token sono 45144.
 
   Le frasi presenti nel secondo corpus sono 2970, mentre i token sono 88774.
 
 - Confronto della lunghezza media delle frasi in token:
 
   La lunghezza media delle frasi nel primo corpus è 20.604290278411685, mentre quella dei token è 3.596314017366649.
 
   La lunghezza media delle frasi nel secondo corpus è 29.89023569023569, mentre quella dei token è 4.149401851893573.
 
 - Confronto del numero di hapax nei primi 500, 1000, 3000 token e in tutto il corpus:
 
    Nel primo corpus gli hapax tra primi 500, 1000, 3000 token e in tutto il corpus sono [203, 350, 730, 3788].
 
    Nel secondo corpus gli hapax tra primi 500, 1000, 3000 token e in tutto il corpus sono [158, 237, 508, 3787].
 
 - Confronto della dimensione del vocabolario e della ricchezza lessicale per proporizoni incrementali di 200 token:
 
    La dimensione del vocabolario e ricch